In [ ]:
https://www.kaggle.com/datasets/chitwanmanchanda/fraudulent-transactions-data?resource=download

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders as ce

In [ ]:
df = pd.read_csv("Fraud.csv")
df.info()

In [ ]:
df.head()

In [3]:
# get column names
df.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [8]:
# Get categories of a column
df['type'].value_counts()

CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: type, dtype: int64

In [11]:
# Select relevant columns
df_v2 = df[['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
       'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud']]

# One hot encoding
df_v3 = pd.get_dummies(df_v2, columns=['type']) #drop_first=True

# Train test split
X = df_v3.drop(['isFraud'], axis=1)
y = df_v3[['isFraud']]
X_train, X_test, y_train, y_test = train_test_split(X,y , random_state=104,test_size=0.25) #, shuffle=True

In [23]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


# determine the number of input features
n_features = X_train.shape[1]

# define model
model = Sequential()
model.add(Dense(10, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.Recall()])

# fit the model
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# evaluate the model
loss, recall = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Recall: {recall:.3f}')



Epoch 1/50
149124/149124 [==============================] - 47s 316us/step - loss: 9.7678 - recall_2: 0.0000e+00
Epoch 2/50
149124/149124 [==============================] - 47s 317us/step - loss: 0.4861 - recall_2: 0.0000e+00
Epoch 3/50
149124/149124 [==============================] - 47s 317us/step - loss: 2.4188 - recall_2: 0.0000e+00
Epoch 4/50
149124/149124 [==============================] - 180s 1ms/step - loss: 4.8731 - recall_2: 0.0000e+00
Epoch 5/50
149124/149124 [==============================] - 47s 314us/step - loss: 1.9832 - recall_2: 0.0000e+00
Epoch 6/50
149124/149124 [==============================] - 47s 316us/step - loss: 0.4708 - recall_2: 0.0000e+00
Epoch 7/50
149124/149124 [==============================] - 47s 318us/step - loss: 1.9873 - recall_2: 0.0000e+00
Epoch 8/50
149124/149124 [==============================] - 47s 316us/step - loss: 3.8848 - recall_2: 0.0000e+00
Epoch 9/50
149124/149124 [==============================] - 48s 320us/step - loss: 0.8476 - recal

In [24]:
# make a prediction
yhat = model.predict(X_test[:10])
print(f'Predicted: {yhat}')

1/1 [==============================] - 0s 34ms/step
Predicted: [[2.1709837e-03]
 [2.1709837e-03]
 [2.1709837e-03]
 [0.0000000e+00]
 [2.1844214e-19]
 [0.0000000e+00]
 [0.0000000e+00]
 [2.1709837e-03]
 [2.1709835e-03]
 [0.0000000e+00]]


# LSTM

In [ ]:
https://archive.ics.uci.edu/dataset/502/online+retail+ii

In [3]:
import pandas as pd
import numpy as np
import datetime as dt
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%2f' % x)

In [4]:
df = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2009-2010")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [5]:
# Drop nulls and convert Customer id to string
df = df.dropna()
df['Customer ID'] = df['Customer ID'].astype(int).astype(str)

In [6]:
# Create y - cancelled transaction
df['cancelled_transaction'] = np.where(df['Invoice'].str.contains('C',na=False),1,0)
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,cancelled_transaction
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950000,13085,United Kingdom,0
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750000,13085,United Kingdom,0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750000,13085,United Kingdom,0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100000,13085,United Kingdom,0
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250000,13085,United Kingdom,0


In [7]:
df['cancelled_transaction'].value_counts()

0    407695
1      9839
Name: cancelled_transaction, dtype: int64

In [8]:
# count of unique values in each column
print(df[['StockCode','Country']].nunique())

StockCode    4031
Country        37
dtype: int64


In [12]:
# Train Test Split

df_grp = pd.DataFrame(df.groupby(['Customer ID']).agg({'cancelled_transaction':['max']})).reset_index()
df_grp.columns = ['Customer ID','Y']
# Train test split
X = df_grp.drop(['Y'], axis=1)
y = df_grp[['Y']]
X_train, X_test, y_train, y_test = train_test_split(X,y , random_state=104,test_size=0.25,stratify=y) #, shuffle=True

Train_cust = X_train['Customer ID'].to_list()
print('len(df): ',len(df))
Train_df = df[df['Customer ID'].isin(Train_cust)]
print('len(Train_df): ',len(Train_df))
Test_df = df[~df['Customer ID'].isin(Train_cust)]
print('len(Test_df): ',len(Test_df))


len(df):  417534
len(Train_df):  326300
len(Test_df):  91234


In [13]:
columns = ['StockCode','Country']
woe_encoder = ce.WOEEncoder(cols=columns)
woe_encoded_train = woe_encoder.fit_transform(Train_df[columns], Train_df['cancelled_transaction']).add_suffix('_woe')
train_df_v2 = Train_df.join(woe_encoded_train)
train_features = train_df_v2[[ 'Quantity', 'InvoiceDate','Price', 'Customer ID',  'StockCode_woe', 'Country_woe']]
train_target = train_df_v2['cancelled_transaction']
print(train_df_v2['cancelled_transaction'].value_counts())
train_features.head()

0    318699
1      7601
Name: cancelled_transaction, dtype: int64


,Quantity,InvoiceDate,Price,Customer ID,StockCode_woe,Country_woe
0,12,2009-12-01 07:45:00,6.950000,13085,1.795105,-0.048949
1,12,2009-12-01 07:45:00,6.750000,13085,2.802890,-0.048949
2,12,2009-12-01 07:45:00,6.750000,13085,2.821527,-0.048949
3,48,2009-12-01 07:45:00,2.100000,13085,-0.196115,-0.048949
4,24,2009-12-01 07:45:00,1.250000,13085,1.413689,-0.048949


In [55]:
woe_encoded_test = woe_encoder.transform(Test_df[columns], Test_df['cancelled_transaction']).add_suffix('_woe')
test_df_v2 = Test_df.join(woe_encoded_test)
test_features = test_df_v2[[ 'Quantity', 'InvoiceDate','Price', 'Customer ID',  'StockCode_woe', 'Country_woe']]
test_target = test_df_v2['cancelled_transaction']
test_df_v2['cancelled_transaction'].value_counts()

0    88996
1     2238
Name: cancelled_transaction, dtype: int64

In [76]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input


# Function to prepare sequences and target
def prepare_sequences(data, sequence_length, x_vars, target_var, date_var, id_var):
    sequences = []
    targets = []
    
    # Group data by customer_id
    grouped = data.groupby(id_var)
    
    for grp_id, group in grouped:
        # Sort transactions by date in ascending order
        sorted_group = group.sort_values(by=date_var, ascending=True)
        
        # Select the last n days of transactions
        last_n_days = sorted_group.iloc[-sequence_length:]
        
        # Fill missing days with zeros
        sequence = last_n_days[x_vars].values
        if len(sequence) < sequence_length:
            sequence = np.vstack([np.zeros((sequence_length - len(sequence), len(x_vars))), sequence])
        
        # Fill missing days with zeros for target data
        target_sequence = last_n_days[target_var].values
        if len(target_sequence) < sequence_length:
            target_sequence = np.pad(target_sequence, (sequence_length - len(target_sequence), 0), mode='constant')
            
        sequences.append(sequence)
        targets.append(target_sequence)
    
    return np.array(sequences), np.array(targets)

# Set the sequence length for the LSTM
sequence_length = 30  # You can adjust this based on the number of days you want to include in the sequence
x_vars = ['Quantity', 'Price', 'StockCode_woe', 'Country_woe']
date_var = 'InvoiceDate'
id_var = 'Customer ID'
target_var = 'cancelled_transaction'


# Prepare sequences and targets for LSTM
X_train, y_train = prepare_sequences(train_df_v2, sequence_length, x_vars, target_var, date_var, id_var)
X_test, y_test = prepare_sequences(test_df_v2, sequence_length, x_vars, target_var, date_var, id_var)


In [77]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(sequence_length, len(x_vars))))
model.add(Dense(sequence_length, activation='sigmoid'))  # Use sequence_length for the Dense layer size


# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["Recall"])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, recall = model.evaluate(X_test, y_test)
print("Test loss:", loss)
print("Test accuracy:", recall)

Epoch 1/50
103/103 [==============================] - 1s 7ms/step - loss: 0.2339 - recall: 0.1488 - val_loss: 0.0979 - val_recall: 0.1363
Epoch 2/50
103/103 [==============================] - 1s 6ms/step - loss: 0.0950 - recall: 0.0573 - val_loss: 0.0883 - val_recall: 0.1622
Epoch 3/50
103/103 [==============================] - 1s 6ms/step - loss: 0.0868 - recall: 0.0961 - val_loss: 0.0805 - val_recall: 0.1951
Epoch 4/50
103/103 [==============================] - 1s 6ms/step - loss: 0.0784 - recall: 0.1922 - val_loss: 0.0724 - val_recall: 0.3478
Epoch 5/50
103/103 [==============================] - 1s 6ms/step - loss: 0.0698 - recall: 0.2895 - val_loss: 0.0656 - val_recall: 0.4477
Epoch 6/50
103/103 [==============================] - 1s 6ms/step - loss: 0.0604 - recall: 0.3397 - val_loss: 0.0564 - val_recall: 0.4454
Epoch 7/50
103/103 [==============================] - 1s 6ms/step - loss: 0.0547 - recall: 0.3616 - val_loss: 0.0514 - val_recall: 0.4724
Epoch 8/50
103/103 [==============